# Librerías utilizadas

In [1]:
%pylab
#pylab.rcParams['figure.figsize'] = (10, 6)
#style.use('default')

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


# Funciones

In [2]:
def GaussianNoise(SNR,Ps,M,N):
    """
    Recibe una potencia de señal Ps y una relación señal-ruido deseada
    SNR y retorna un vector Nu correspondiente a muestras del ruido 
    blanco gaussiano a sumarle a la señal
    """
    mean_W = 0
    var_W = Ps/(10**(SNR/10))
    std_W = np.sqrt(var_W)
    W = np.asmatrix(np.random.normal(mean_W, std_W, size=(M,N)))
    return (W)

def print_matrix(list_of_list):
    number_width = len(str(max([max(i) for i in list_of_list])))
    cols = max(map(len, list_of_list))
    output = '+'+('-'*(number_width+2)+'+')*cols + '\n'
    for row in list_of_list:
        for column in row:
            output += '|' + ' {:^{width}d} '.format(column, width = number_width)
        output+='|\n+'+('-'*(number_width+2)+'+')*cols + '\n'
    return output

def plotanim(Data,X,step=0.02,ylim=[]):
    plt.ion()
    fig, ax = plt.subplots(1,1)
    plt.ylim((10 ** -3, Data.max()))
    plt.yscale('log')
    line, = ax.plot(X,Data[:,0])
    fig.canvas.draw()
    for i in range(Data.shape[1]):
        plt.pause(step)
        line.set_ydata(Data[:,i])
        fig.canvas.draw()
    plt.ioff()

# Definición de constantes y unidades

In [3]:
kHz = 10 ** 3
MHz = 10 ** 6
GHz = 10 ** 9
ms = 10 ** -3
c = 3 * 10 ** 8
km = 10 ** 3

# Definición de variables

In [4]:
Mx = 4  # Número de sensores en la dirección x
My = 4  # Número de sensores en la dirección y
M = Mx * My # Número de sensores
D = 4  # Número de señales
N = 100 # Número de snapshots

fs = 64 * MHz

T = 4 * ms # Tiempo de toma de muestras
t0 = 0

theta_deg = np.array([-60,30,57,-85])  #DOA en grados
phi_deg = np.array([45,32,-52,-69])
theta = theta_deg * np.pi / 180           #DOA en radianes
phi = phi_deg * np.pi / 180           #DOA en radianes

# Defición de señal y array de sensores
Fc = np.array([436*MHz,436*MHz,436*MHz,436*MHz]) # Frecuencia de portadora de la señal transmitida 1
lambda_c = c / Fc
d = lambda_c[0] / 2  #Separación entre sensores

s_amp = np.array([100,50,200,250])
s_freq = np.array([440,3000,40000,10000])

SNR = 7

# Definición de señales y muestras

## Definición del vector aleatorio de tiempo de toma de snapshots

In [5]:
t = np.random.randint(0, int(T * fs), size=N) * 1 / fs  # Vector de tiempos de muestreo (tomas de snapshots)

## Genero la matriz F de tamaño (D,N)

In [6]:
lambda_c = c / Fc
K = 2 * np.pi / (lambda_c)
F = np.asmatrix(np.empty((D, N)))
for i in range(0, D):
    F[i,:] = s_amp[i] * np.cos(2 * np.pi * s_freq[i] * (t+t0))

## Genero la matriz A de tamaño (M,D) considerando $g(\theta)=1 \quad \forall \  \theta$

In [7]:
A = np.asmatrix(np.empty((M, D), dtype='complex'))
for i in range(0, Mx):
    for j in range(0,My):
        for k in range(0, D):
            A[Mx*j+i, k] = np.e ** (-1j * (i * K[k] * d * sin(theta[k]) * cos(phi[k]) + j * K[k] * d * sin(theta[k]) * sin(phi[k])))

## Genero el vector W de tamaño (M,N) de ruido a partir de una SNR dada

In [8]:
Ps = 0
for i in range(0,D):
    Ps = Ps + (s_amp[i]**2)/2
W = GaussianNoise(SNR,Ps,M,N)

## Genero el vector de muestras $X=A \times F + W$ de tamaño (M,1,N)

In [9]:
X = np.asmatrix(np.empty((M,N),dtype=complex))
X_matrix = np.asmatrix(np.empty((M,1)),dtype=complex)
for n in range (0,N):
    F_matrix = F[:,n]
    W_matrix = W[:,n]
    X_matrix = A @ F_matrix + W_matrix
    X[:, n] = X_matrix

# Calculo el vector de covarianza S

In [10]:
S = np.asmatrix(np.empty((M,M),dtype=complex))

for n in range (0,N):
    X_matrix=np.asmatrix(X[:,n])
    S = S + (1/N)*(X_matrix @ X_matrix.H)

# Encuentro autovalores y autovectores S y el autovalor mínimo $\lambda_{min}$ tal que $|S-\lambda_{min}\cdot S_0|=0$

In [11]:
[aval, avec] = eig(S)
S0 = np.identity(M)
p = aval.argsort()
aval=np.abs(aval[p])
avec=avec[:,p]
aval_min = aval[0]

# Encuentro la multiplicidad Q de $\lambda_min$ y el número estimado de señales $\hat{D}$

In [12]:
Q = 0
umbral = 2 * aval_min
for i in range (0,M):
    if aval[i]-aval_min<umbral:
        Q += 1
D_est = M - Q

# Formo la matriz de subespacio de ruido $E_N$

In [13]:
EN = np.asmatrix(avec[:,0:Q])


# Evalúo la función $P_{MU}$ para distintas frecuencias de portadora y distintos ángulos de arribo

In [14]:
K_est = 2 * np.pi * Fc/c
theta_est = np.linspace(-pi/2, pi/2 ,num=50)
phi_est = np.linspace(-pi/2, pi/2 ,num=50)

a = np.asmatrix(np.empty((M, 1), dtype=complex))
P_MU = np.empty((theta_est.size,phi_est.size, K_est.size), dtype=complex)


for lx in range (0,theta_est.size):
    for ly in range (0,phi_est.size):
        for k in range(0, K_est.size):
            for i in range(0, Mx):
                for j in range(0,My):
                    a[Mx*j+i] = np.e ** (-1j * (i * K_est[k] * d * sin(theta_est[lx]) * cos(phi_est[ly]) + j * K_est[k] * d * sin(theta_est[lx]) * sin(phi_est[ly])))
                    P_MU[lx, ly, k] = 1 / (a.H @ EN @ EN.H @ a)

## Ploteo $P_{MU}$ en 3D

In [15]:
x = theta_est*180/pi
y = phi_est*180/pi
figure()
imshow(log10(abs(P_MU[:,:,0])),extent=(x.min(),x.max(),y.max(),y.min()),cmap='jet')
xlabel(r'$\theta$')
ylabel(r'$\phi$')
show()

# Creo funciones para automatizar la creación de las muestras y el algoritmo music

In [16]:
def doamusic_samples(Fc,s_amp,s_freq,t,t0,d,theta,phi,Mx,My,D,SNR):
    M = Mx * My
    # Definición de señales y muestras
    # Genero la matriz F de tamaño (D,N)
    lambda_c = c / Fc
    K = 2 * np.pi / (lambda_c)
    F = np.asmatrix(np.empty((D, N)))
    for i in range(0, D):
        F[i,:] = s_amp[i] * np.cos(2 * np.pi * s_freq[i] * (t+t0))

    # Genero la matriz A de tamaño (M,D) considerando $g(\theta)=1 \quad \forall \  \theta$
    A = np.asmatrix(np.empty((M, D), dtype='complex'))

    for i in range(0, Mx):
        for j in range(0,My):
            for k in range(0, D):
                A[Mx*j+i, k] = np.e ** (-1j * (i * K[k] * d * sin(theta[k]) * cos(phi[k]) + j * K[k] * d * sin(theta[k]) * sin(phi[k])))

    # Genero el vector W de tamaño (M,1,N) de ruido $W\sim N(0,1)$
    Ps = 0
    for i in range(0,D):
        Ps = Ps + (s_amp[i]**2)/2
    W = GaussianNoise(SNR,Ps,M,N)

    # Genero el vector de muestras $X=A \times F + W$ de tamaño (M,1,N)
    X = np.asmatrix(np.empty((M,N),dtype=complex))
    X_matrix = np.asmatrix(np.empty((M,1)),dtype=complex)
    for n in range (0,N):
        F_matrix = F[:,n]
        W_matrix = W[:,n]
        X_matrix = A @ F_matrix + W_matrix
        X[:, n] = X_matrix

    return X

def doamusic_estimation(X, K_est, theta_est, phi_est, Mx, My):
    M = Mx * My
    N = X.shape[1]

    #%% Calculo el vector de covarianza S
    S = np.asmatrix(np.empty((M,M),dtype=complex))
    for n in range (0,N):
        X_matrix=np.asmatrix(X[:,n])
        S = S + (1/N)*(X_matrix @ X_matrix.H)

    # Encuentro autovalores y autovectores S y el autovalor mínimo $\lambda_{min}$ tal que $|S-\lambda_{min}\cdot S_0|=0$
    [aval, avec] = eig(S)
    S0 = np.identity(M)
    p = aval.argsort()
    aval=np.abs(aval[p])
    avec=avec[:,p]
    aval_min = aval[0]

    # Encuentro la multiplicidad Q de $\lambda_min$ y el número estimado de señales $\hat{D}$
    Q = 0
    umbral = 2 * aval_min
    for i in range (0,M):
        if aval[i]-aval_min<umbral:
            Q += 1
    D_est = M - Q

    # Formo la matriz de subespacio de ruido $E_N$
    EN = np.asmatrix(avec[:,0:Q])

    #%% Evalúo la función $P_{MU}$ para distintas frecuencias de portadora y distintos ángulos de arribo

    a = np.asmatrix(np.empty((M, 1), dtype=complex))
    P_MU = np.empty((theta_est.size,phi_est.size, K_est.size), dtype=complex)


    for lx in range (0,theta_est.size):
        for ly in range (0,phi_est.size):
            for k in range(0, K_est.size):
                for i in range(0, Mx):
                    for j in range(0,My):
                        a[Mx*j+i] = np.e ** (-1j * (i * K_est[k] * d * sin(theta_est[lx]) * cos(phi_est[ly]) + j * K_est[k] * d * sin(theta_est[lx]) * sin(phi_est[ly])))  
                        P_MU[lx, ly, k] = 1 / (a.H @ EN @ EN.H @ a)
    return P_MU

# Script

In [20]:
Mx = 4  # Número de sensores en la dirección x
My = 4  # Número de sensores en la dirección y
D = 4  # Número de señales
N = 100 # Número de snapshots

fs = 64 * MHz

T = 5 * ms # Tiempo de toma de muestras
t0 = 0

theta_deg = np.array([45,45,45,45])  #DOA en grados
phi_deg = np.array([-45,45,135,-135])
theta = theta_deg * np.pi / 180           #DOA en radianes
phi = phi_deg * np.pi / 180           #DOA en radianes

# Defición de señal y array de sensores
Fc = np.array([436*MHz,436*MHz,436*MHz,436*MHz]) # Frecuencia de portadora de la señal transmitida 1
lambda_c = c / Fc
d = lambda_c[0] / 2  #Separación entre sensores

s_amp = np.array([100,100,100,100])
s_freq = np.array([440,3000,40000,10000])

SNR = 7

t = np.random.randint(0, int(T * fs), size=N) * 1 / fs  # Vector de tiempos de muestreo (tomas de snapshots)

K_est = 2 * np.pi * Fc/c
theta_est = np.linspace(0, pi/2 ,num=50)
phi_est = np.linspace(-pi, pi ,num=50)

In [21]:
X = doamusic_samples(Fc, s_amp, s_freq, t,0, d, theta,phi, Mx,My, D, SNR)
P_MU = doamusic_estimation(X, K_est, theta_est, phi_est , Mx, My)

In [22]:
x = theta_est*180/pi
y = phi_est*180/pi
figure()
imshow(log10(abs(P_MU[:,:,0])),extent=(x.min(),x.max(),y.max(),y.min()),cmap='jet',aspect="auto")
xlabel(r'$\theta$')
ylabel(r'$\phi$')
show()